## Importing necessary packages

In [ ]:
import mickey
import mickey.mickey
import mickey.plot
import tqdm
import numpy as np
import scipy.optimize
import seaborn

## Opening directory of interest

In [ ]:
#%cd /home/artur/Desktop/Torus_pn_2D_cooling/Simulations/rho1.5e-6/
%cd /home/artur/Desktop/Torus_pn_2D_cooling/Simulations/rho1e-6/
#%cd /home/artur/Desktop/Torus_pn_2D_cooling/Simulations/rho5e-7/
#%cd /home/artur/Desktop/Torus_pn_2D_cooling/Simulations/rho3e-7/
#%cd /home/artur/Desktop/Torus_pn_2D_cooling/Simulations/rho5e-8/
#%cd /home/artur/Desktop/Torus_pn_2D_cooling/Simulations/rho1e-8/
#%cd /home/artur/Desktop/Torus_pn_2D_cooling/Simulations/rho5e-9/

## Load up the plotting methods

In [ ]:
%pylab inline

## Parameters needed

In [ ]:
#Mass of black hole in solar mass
mbh = 10.0

#units of sim
unit_length = 2.969707e6
unit_velocity = 2.1198528e10

#MOST IMPORTANT UNIT TO MY WORK
unit_density = 3e-7

#Eddington accretion rate for mbh in cgs using efficiency of 0.1
eddingtonac = 1.39916e18 * mbh

#Conversion of mdot from code units to cgs
mdot_conv = unit_length*unit_length*unit_density*unit_velocity

#Conversion of time to orbital time in 100Rs
torb = 6220.8

#Conversion of time to GM/c3
tm = 2.828427

## Calculating accretion rate

In [ ]:
# will hold mdot-values at the inner boundary
mdot=[]

# simulation time in orbits and unit of GM/C3
t=[]
tgmc3 = []

for i in tqdm.tqdm(range(2634,2989)):
    # reads data
    p=mickey.mickey.Pluto(i, stdout=False)
    
    mdot.append(p.mdotr(0)/eddingtonac*mdot_conv)
    t.append(p.t/torb)
    tgmc3.append(p.t*tm)

t,mdot,tgmc3=array(t),array(mdot),array(tgmc3)

In [ ]:
#Gaussian
def gaussian(x, amp1,cen1,sigma1):
    return amp1*(1/(sigma1*(np.sqrt(2*np.pi))))*(np.exp(-((x-cen1)**2)/(2*(sigma1)**2)))

#Initial and final snapshot analysed
snap_i = 2423
snap_f = 2424

for k in tqdm.tqdm(range(snap_i, snap_f)):
    
    p=mickey.mickey.Pluto(k,stdout=False)
    c=p.regridFast(2000,xlim=100.)
    
    #Holds aspect ratio
    hr = []

    xx = c.rho.T[600][100:1900]
    yy = c.x2[100:1900]
    n = len(yy)
    mean = sum(yy * xx)/n
    sigma = sum(xx*(yy-mean)**2)/n
    amp = xx.max()

    popt, pcov = scipy.optimize.curve_fit(gaussian, yy, xx, p0=[amp, mean, sigma], maxfev=100000)

In [ ]:
fit = gaussian(c.x2[100:1900], popt[0], popt[1], popt[2])

plot(c.x2[100:1900], c.rho.T[600][100:1900], label="Data")
plot(c.x2[100:1900], fit, label="Gaussian fit")
plot(np.ones(2)*(popt[1]-popt[2]),[np.amin(fit), np.amax(fit)], ":k")
plot(np.ones(2)*(popt[1]+popt[2]),[np.amin(fit), np.amax(fit)], ":k")
fill_betweenx([np.amin(fit), np.amax(fit)], popt[1]-popt[2], popt[1]+popt[2], alpha="0.1", color="black")

plt.xlim(-2,3)

#title("Gaussian fit", fontsize=16)

#xscale('log')
#yscale('log')

xlabel("$Z/R_s$", fontsize=15)
ylabel(r"$\rho$ (code units)", fontsize=15)

plt.legend(fontsize=13)

tick_params(axis='both', which='major', labelsize=13)
tick_params(axis='both', which='minor', labelsize=12)
tight_layout()

plt.savefig('/home/artur/Desktop/Torus_pn_2D_cooling/Simulations/fitting_gaussian.png', dpi=300)
plt.clf()
plt.close('all')
#plt.show()

## Calculate scale height by fitting a gaussian to each radii cell

In [ ]:
#getting scale height H by fitting a gaussian in the density function of each x, rho(y)
#Holds the truncation radius for each snapshot
Rtr = []

#Range to calculate the truncation radius
init = 0
final = 80

#Gaussian
def gaussian(x, amp1,cen1,sigma1):
    return amp1*(1/(sigma1*(np.sqrt(2*np.pi))))*(np.exp(-((x-cen1)**2)/(2*(sigma1)**2)))

#Initial and final snapshot analysed
snap_i = 2634
snap_f = 2989

for k in tqdm.tqdm(range(snap_i, snap_f)):
    
    p=mickey.mickey.Pluto(k,stdout=False)
    c=p.regridFast(2000,xlim=100.)
    
    #Holds aspect ratio
    hr = []

    for j in range(30, 1900):
        xx = c.rho.T[j][100:1900]
        yy = c.x2[100:1900]
        n = len(yy)
        mean = sum(yy * xx)/n
        sigma = sum(xx*(yy-mean)**2)/n
        amp = xx.max()

        popt, pcov = scipy.optimize.curve_fit(gaussian, yy, xx, p0=[amp, mean, sigma], maxfev=100000)

        hr.append(popt[2]/c.x1[j])

    #GETTING Rtr BY TAKING A RANGE OF H/R AND TAKING THE MEAN TO BE LESS THAN 0.015
    hri = hr[init:final]
    media = np.mean(hri)
    for i in range(1870):
        if media < 0.015:
            Rtr.append(np.mean(c.x1[init+i+30:final+i+30]))
            break
        elif i + final < 1870:
            del hri[0]
            hri.append(hr[i+final])
            media = np.mean(hri)
        else:
            break

## Calculate Rtr fitting -a * Tanh(x - b) + c

In [ ]:
#getting scale height H by fitting a gaussian in the density function of each x, rho(y)
#Holds the truncation radius for each snapshot
Rtr = []

snap_i = 2200
snap_f = 2201

#Gaussian
def gaussian(x, amp1,cen1,sigma1):
    return amp1*(1/(sigma1*(np.sqrt(2*np.pi))))*(np.exp(-((x-cen1)**2)/(2*(sigma1)**2)))

#Hiperbolic Tangent
def minus_tanh(x, a1, a2, a3):
     return (-a1 * np.tanh(x - a2) + a3)

for k in tqdm.tqdm(range(snap_i, snap_f)):
    
    p=mickey.mickey.Pluto(k,stdout=False)
    c=p.regridFast(2000,xlim=100.)
    hr = []
    
    for j in range(30, 1900):
        xx = c.rho.T[j][100:1900]
        yy = c.x2[100:1900]
        n = len(yy)
        mean = sum(yy * xx)/n
        sigma = sum(xx*(yy-mean)**2)/n
        amp = xx.max()

        popt, pcov = scipy.optimize.curve_fit(gaussian, yy, xx, p0=[amp, mean, sigma], maxfev=100000)

        hr.append(popt[2]/c.x1[j])
    
    #Initial guesses
    a1 = 0.06
    a2 = 10.
    a3 = 0.06
    
    fit, cov = scipy.optimize.curve_fit(minus_tanh, c.x1[30:1900], hr, p0=[a1, a2, a3], maxfev=10000)
    Rtr.append(fit[1])

## Use both methods

In [ ]:
#getting scale height H by fitting a gaussian in the density function of each x, rho(y)
#Holds the truncation radius for each snapshot
Rtr1 = []
Rtr2 = []

#Range to calculate the truncation radius
init = 0
final = 50

#Gaussian
def gaussian(x, amp1,cen1,sigma1):
    return amp1*(1/(sigma1*(np.sqrt(2*np.pi))))*(np.exp(-((x-cen1)**2)/(2*(sigma1)**2)))

#Hiperbolic Tangent
def minus_tanh(x, a1, a2, a3):
     return (-a1 * np.tanh(x - a2) + a3)

#Initial and final snapshot analysed
snap_i = 2400
snap_f = 2401

for k in tqdm.tqdm(range(snap_i, snap_f)):
    
    p=mickey.mickey.Pluto(k,stdout=False)
    c=p.regridFast(2000,xlim=100.)
    
    #Holds aspect ratio
    hr = []

    for j in range(30, 1900):
        xx = c.rho.T[j][100:1900]
        yy = c.x2[100:1900]
        n = len(yy)
        mean = sum(yy * xx)/n
        sigma = sum(xx*(yy-mean)**2)/n
        amp = xx.max()

        popt, pcov = scipy.optimize.curve_fit(gaussian, yy, xx, p0=[amp, mean, sigma], maxfev=100000)

        hr.append(popt[2]/c.x1[j])
    
    #Initial guesses
    a1 = 0.06
    a2 = 10.
    a3 = 0.06
    
    fit, cov = scipy.optimize.curve_fit(minus_tanh, c.x1[30:1900], hr, p0=[a1, a2, a3], maxfev=10000)
    Rtr1.append(fit[1])

    #GETTING Rtr BY TAKING A RANGE OF H/R AND TAKING THE MEAN TO BE LESS THAN 0.015
    hri = hr[init:final]
    media = np.mean(hri)
    for i in range(1870):
        if media < 0.015:
            Rtr2.append(np.mean(c.x1[init+i+30:final+i+30]))
            break
        elif i + final < 1870:
            del hri[0]
            hri.append(hr[i+final])
            media = np.mean(hri)
        else:
            break

## Plot every fit of Rtr to analyse its goodness

In [ ]:
#getting scale height H by fitting a gaussian in the density function of each x, rho(y)
#Holds the truncation radius for each snapshot
Rtr = []
rtr = []

#Range to calculate the truncation radius
init = 0
finala = 80
finalb = 5

#Gaussian
def gaussian(x, amp1,cen1,sigma1):
    return amp1*(1/(sigma1*(np.sqrt(2*np.pi))))*(np.exp(-((x-cen1)**2)/(2*(sigma1)**2)))

#Initial and final snapshot analysed
snap_i = 2440
snap_f = 2441

for k in tqdm.tqdm(range(snap_i, snap_f)):
    
    p=mickey.mickey.Pluto(k,stdout=False)
    c=p.regridFast(2000,xlim=100.)
    
    #Holds aspect ratio
    hr = []

    for j in range(30, 1900):
        xx = c.rho.T[j][100:1900]
        yy = c.x2[100:1900]
        n = len(yy)
        mean = sum(yy * xx)/n
        sigma = sum(xx*(yy-mean)**2)/n
        amp = xx.max()

        popt, pcov = scipy.optimize.curve_fit(gaussian, yy, xx, p0=[amp, mean, sigma], maxfev=100000)

        hr.append(popt[2]/c.x1[j])

    #GETTING Rtr BY TAKING A RANGE OF H/R AND TAKING THE MEAN TO BE LESS THAN 0.015
    hri = hr[init:finala]
    media = np.mean(hri)
    for i in range(1870):
        if media < 0.015:
            Rtr.append(np.mean(c.x1[init+i+30:finala+i+30]))
            break
        elif i + final < 1870:
            del hri[0]
            hri.append(hr[i+finala])
            media = np.mean(hri)
        else:
            break

    hri = hr[init:finalb]
    media = np.mean(hri)
    for i in range(1870):
        if media < 0.015:
            rtr.append(np.mean(c.x1[init+i+30:finalb+i+30]))
            break
        elif i + final < 1870:
            del hri[0]
            hri.append(hr[i+finalb])
            media = np.mean(hri)
        else:
            break
    

    
    figure(figsize=(8,4))
    
    #plot(rtr[k - snap_i]*ones(2), [0,0.15], ":k", label="$\Delta r = %0.2f$" % ( Rtr[k - snap_i] - rtr[k - snap_i]))
    plot(Rtr[k - snap_i]*ones(2), [0,0.15],'--k', label='Rtr = %0.2f' % Rtr[k - snap_i])
    plot(c.x1[30:1900], hr, lw=3, color='b', label='Gaussian Fit')
    #plot(c.x1[30:1900], minus_tanh(c.x1[30:1900], fit[0], fit[1], fit[2]), lw=3, color='r', label='$Tanh (x)$ Fit')

    title("Aspect Ratio", fontsize=16)

    xscale('log')
    #yscale('log')

    xlabel("$R/R_s$", fontsize=15)
    ylabel("$H/R$", fontsize=15)

    plt.legend(fontsize=13)

    tick_params(axis='both', which='major', labelsize=13)
    tick_params(axis='both', which='minor', labelsize=12)
    tight_layout()

    plt.savefig('/home/artur/Desktop/Torus_pn_2D_cooling/Simulations/aspect_ratio.png', dpi=300)
    plt.clf()
    plt.close('all')
    #plt.show()

In [ ]:
print(c.x1[rtr1+8])

## Print Rtr and mdot with uncertainties

In [ ]:
#print (Rtr)
#print (Rtr1, Rtr2)
print (np.mean(Rtr), np.std(Rtr))
print (np.mean(abs(mdot)), np.std(abs(mdot)))

## Saving/Loading array 

In [ ]:
np.savez('data1.npz', abs(mdot), Rtr)

In [ ]:
data = np.load('data.npz')
#print (data['arr_0'])
mdot = data['arr_0']
Rtr = data['arr_1']

In [ ]:
print((max(Rtr)-min(Rtr))/sqrt(len(Rtr)))
print(np.median(Rtr))

## Plotting Rtr vs Mdot

In [ ]:
#figure(figsize=(8,6))

seaborn.set(style="white", color_codes=True)
g = seaborn.jointplot(x=mdot, y=Rtr, kind='scatter', color = 'xkcd:blue', space=0, ratio=6, size=7, stat_func=None,
                     marginal_kws=dict(bins=15, rug=False))
g = g.set_axis_labels("$\dot{M}/\dot{M}_{\\rm Edd}$", "$R_{\\rm tr}/R_s$")
g = g.plot_joint(seaborn.kdeplot, zorder=1, n_levels=8)
plt.text(0.055,47.2, '$R_{\\rm tr}/R_s = %0.1f \pm %0.1f$\n$\dot{M}/\dot{M}_{\\rm Edd} = %0.2f \pm %0.2f$' 
         % (np.median(Rtr), np.std(Rtr),np.mean(mdot), np.std(mdot)), fontsize = 18,
            color='Black', fontstyle='italic', bbox=dict(boxstyle="square", lw=1.1,
                   ec=(0, 0, 0), fc=(1, 1, 1)))
g = g.savefig('Rtr_vs_mdot_3e-7.png', dpi=300)

#errorbar(np.mean(abs(mdot)), np.median(Rtr), np.std(Rtr), np.std(abs(mdot)), marker='o', color='r', markersize=10, capsize=10)
#scatter(np.mean(abs(mdot)), np.median(Rtr), marker='o', c='r', label='$R_{\\rm tr}/R_s = %0.1f \pm %0.1f$\n $\dot{M}/\dot{M}_{\\rm Edd} = %0.2f \pm %0.2f$' % (np.median(Rtr), np.std(Rtr),np.mean(abs(mdot)), np.std(abs(mdot))))
#scatter(abs(mdot), Rtr, marker='o', c='b')

#plots.jointplot(mdot, Rtr, xlabel="$R_{\\rm tr}/R_s$", ylabel="$\dot{M}/\dot{M}_{\\rm Edd}$")

#title("Truncation radius vs Accretion rate", fontsize=16)

#ylabel("$R_{\\rm tr}/R_s$", fontsize=15)
#xlabel("$\dot{M}/\dot{M}_{\\rm Edd}$", fontsize=15)

#plt.legend(fontsize=13)

#tick_params(axis='both', which='major', labelsize=13)
#tick_params(axis='both', which='minor', labelsize=12)
#tight_layout()

#plt.show()
#plt.savefig('Rtr_vs_mdot_3e-7.png', dpi=300)
plt.clf()
plt.close('all')

## Plotting Aspect Ratio vs R/Rs with Rtr

In [ ]:
figure(figsize=(8,4))

plot(Rtr*ones(2), np.linspace(0,0.15, 2),'--k', label='Rtr = %0.2f' % Rtr[0])
plot(c.x1[30:1900], hr, lw=3, color='b', label='Gaussian Fit')
#plot(c.x1[30:1900], minus_tanh(c.x1[30:1900], fit[0], fit[1], fit[2]), lw=3, color='r', label='$Tanh (x)$ Fit')

title("Aspect Ratio", fontsize=16)

#xscale('log')
#yscale('log')

xlabel("$R/R_s$", fontsize=15)
ylabel("$H/R$", fontsize=15)

plt.legend(fontsize=13)

tick_params(axis='both', which='major', labelsize=13)
tick_params(axis='both', which='minor', labelsize=12)
tight_layout()

plt.savefig('aspect_ratio.png', dpi=300)
plt.clf()
plt.close('all')